## Install latest dataflows framework

In [ ]:
%%bash
python3 -m pip install dataflows==0.0.64 sqlalchemy==1.3.11 knesset-data==2.1.5 tableschema-sql==1.3.1

Restart the kernel

## Setup DB connection

Should connect to the redash_reader read-only user

In [1]:
import getpass

# postgresql://redash_reader:SECRET_PASSWORD@DB_HOST:5432/postgres

redash_reader_connection_string = getpass.getpass('redash_reader connection string:')

redash_reader connection string: ································································································


In [2]:
from sqlalchemy import create_engine

redash_reader_db_engine = create_engine(redash_reader_connection_string)

In [3]:
def stream_from_db(sql):
    conn = redash_reader_db_engine.connect()
    proxy = conn.execution_options(stream_results=True).execute(sql)
    while True:
        rows = proxy.fetchmany(100)
        if len(rows) == 0:
            break
        for row in rows:
            yield dict(zip(proxy.keys(), row))
    conn.close()

In [4]:
for row in stream_from_db('select * from people_committees_meeting_speaker_stats limit 5'):
    print(row)

{'CommitteeSessionID': Decimal('100018'), 'parts_crc32c': 'ARQVTQ==', 'part_index': Decimal('0'), 'header': None, 'body_length': Decimal('218'), 'body_num_words': Decimal('26'), 'part_categories': None, 'name_role': None, 'mk_individual_id': None, 'mk_individual_faction': None}
{'CommitteeSessionID': Decimal('100018'), 'parts_crc32c': 'ARQVTQ==', 'part_index': Decimal('1'), 'header': 'סדר היום', 'body_length': Decimal('183'), 'body_num_words': Decimal('29'), 'part_categories': None, 'name_role': None, 'mk_individual_id': None, 'mk_individual_faction': None}
{'CommitteeSessionID': Decimal('100018'), 'parts_crc32c': 'ARQVTQ==', 'part_index': Decimal('10'), 'header': 'אבי פלדמן', 'body_length': Decimal('13'), 'body_num_words': Decimal('2'), 'part_categories': 'משפטן,משרד ממשלתי אחר', 'name_role': 'עו"ד אבי פלדמן - ס\' יועמ"ש, משרד התמ"ת', 'mk_individual_id': None, 'mk_individual_faction': None}
{'CommitteeSessionID': Decimal('100018'), 'parts_crc32c': 'ARQVTQ==', 'part_index': Decimal('10

## Generate the detailed package

output data is spread over the following directory structure:
    
```
./data/detailed_speaker_parts/Knesset{KnessetNum}/Committee{committee_name}/datapackage.json
```

In [9]:
%%bash
rm -rf data/detailed_speaker_parts

### Split sessions to groups

In [10]:
from collections import defaultdict
from dataflows import Flow, printer


SPEAKER_STATS_PART_GROUPS = {}
SESSIONS = {}


for session in stream_from_db("""
select "CommitteeSessionID", "KnessetNum", "committee_name", "StartDate" 
from committees_kns_committeesession 
where "KnessetNum" > 14 and "KnessetNum" < 22
and related_to_legislation = true
order by "KnessetNum", "committee_name", "StartDate"
"""):
    SESSIONS[session['CommitteeSessionID']] = session
    key = '{}-{}'.format(
        session['KnessetNum'],
        session['committee_name'],
    )
    subkey = '{}'.format(
        session['StartDate'].year,
#         session['StartDate'].month
    )
    if key not in SPEAKER_STATS_PART_GROUPS:
        SPEAKER_STATS_PART_GROUPS[key] = {}
    if subkey not in SPEAKER_STATS_PART_GROUPS[key]:
        SPEAKER_STATS_PART_GROUPS[key][subkey] = set()
    SPEAKER_STATS_PART_GROUPS[key][subkey].add(session['CommitteeSessionID'])

    
def _get_keys_num_session_ids():
    max_subkey_session_ids = 0
    for key, subkeys in SPEAKER_STATS_PART_GROUPS.items():
        num_session_ids = 0
        for subkey, session_ids in subkeys.items():
            num_session_ids += len(session_ids)
            if len(session_ids) > max_subkey_session_ids:
                max_subkey_session_ids = len(session_ids)
        yield {'key': key, 'num_session_ids': num_session_ids}
    print('max_subkey_session_ids = {}'.format(max_subkey_session_ids))
    assert max_subkey_session_ids < 500
    

Flow(
    _get_keys_num_session_ids(),
    printer(tablefmt='html', num_rows=1)
).process()[1]

max_subkey_session_ids = 260


#,key (string),num_session_ids (integer)
1,15-הוועדה המיוחדת לענייני מחקר ופיתוח מדעי וטכנולוגי,20
2,15-הוועדה המיוחדת לקידום מעמד הילד,49
...,,
266,21-ועדת הכספים,13


{}

In [11]:
from dataflows import Flow, load, printer, add_field, dump_to_path, sort_rows, update_resource, checkpoint
from collections import defaultdict
from fuzzywuzzy import fuzz
import traceback
import json
import os
import datetime
import traceback


LIMIT_ROWS_PER_PART_GROUP=None

# FILTER_PART_GROUPS=lambda session, stats: session['KnessetNum'] == 20 and stats['num_sessions'] <= 50
# FILTER_PART_GROUPS=None

# מס כנסת, תאריך, ועדה, הצח פרטית או ממשלתית (אם קיים), שם החוק (אם קיים), שם הדובר, שיוך הדובר, מספר הפעמים שקיבל זכות דיבור,
# אחוז מספר המילים שאמר מסך כל המילים שנאמרו על ידי סך כל הדוברים בפרוטוקול, אורך זכות הדיבור הארוכה ביותר שלו# # .


def _get_part_group_target_path(part_group_key):
    tmp = part_group_key.split('-')
    KnessetNum = tmp[0]
    committee_name = '-'.join(tmp[1:])
    return 'data/detailed_speaker_parts/Knesset{}/{}/'.format(KnessetNum, committee_name[:60])


def _dump_sessions_parts_to_path(part_group_key, parts):
    print('dumping part_group_key {} with {} parts'.format(part_group_key, len(parts)))
    # first_session = SESSIONS[parts[0]['CommitteeSessionID']]
    target_path = _get_part_group_target_path(part_group_key)
    parts_session_ids = set()
    for part in parts:
        session = SESSIONS[part['CommitteeSessionID']]
        if part['CommitteeSessionID'] not in parts_session_ids:
            parts_session_ids.add(part['CommitteeSessionID'])
            session['headers'] = {}
            session['headers_stats'] = {
                'num_words': 0,
                'num_parts': 0
            }
        header = part['header']
        if header is not None and len(header.strip()) > 2:
            if not session['headers'].get(header):
                session['headers'][header] = {
                    'CommitteeSessionID': part['CommitteeSessionID'],
                    'header': header,
                    'Note': part['Note'],
                    'topics': part['topics'],
                    'KnessetNum': part['KnessetNum'],
                    'StartDate': part['StartDate'],
                    'committee_name': part['committee_name'],
                    'bill_types': part['bill_types'],
                    'bill_names': part['bill_names'],
                    'categories': set(),
                    'num_words': 0,
                    'num_parts': 0,
                    'longest_part_num_words': 0,
                    'name_roles': set(),
                    'mk_factions': set(),
                }
            header = session['headers'][header]
            if part['part_categories']:
                for part_category in part['part_categories'].split(','):
                    header['categories'].add(part_category)
            if part['name_role']:
                header['name_roles'].add(part['name_role'])
            if part['mk_individual_faction']:
                header['mk_factions'].add(part['mk_individual_faction'])
            header['num_parts'] += 1
            session['headers_stats']['num_parts'] += 1
            header['num_words'] += part['body_num_words']
            session['headers_stats']['num_words'] += part['body_num_words']
            if part['body_num_words'] > header['longest_part_num_words']:
                header['longest_part_num_words'] = part['body_num_words']    
    num_headers = 0
    num_categories = 0
    for session_id in parts_session_ids:
        session = SESSIONS[session_id]
        session['categories'] = {}
        for header in session['headers'].values():
            num_headers += 1
            header['categories'] = [cat for cat in header['categories'] if cat]
            header['name_roles'] = list(header['name_roles'])
            header['mk_factions'] = list(header['mk_factions'])
            header['session_num_words'] = session['headers_stats']['num_words']
            header['session_num_parts'] = session['headers_stats']['num_parts']
            try:
                header['percent_num_words'] = round(header['num_words'] / header['session_num_words'] * 100, 2)
            except Exception:
                traceback.print_exc()
                print('error getting percent_num_words for session {} (num_words = {}, session_num_words = {})'.format(session_id, header['num_words'], header['session_num_words']))
                header['percent_num_words'] = 0
            category = None
            for aggcat in ['chairperson', 'mk', 'משרד המשפטים', 'משרד ממשלתי אחר', 'legal_advisor', 'משפטן']:
                if aggcat in header['categories']:
                    category = aggcat
                    break
            if category:
                num_categories += 1
                category = session['categories'].setdefault(category, {
                    'CommitteeSessionID': session_id,
                    'Note': header['Note'],
                    'topics': header['topics'],
                    'KnessetNum': header['KnessetNum'],
                    'StartDate': header['StartDate'],
                    'committee_name': header['committee_name'],
                    'bill_types': header['bill_types'],
                    'bill_names': header['bill_names'],
                    'category': category,
                    'num_words': 0,
                    'num_parts': 0,
                    'longest_part_num_words': 0,
                    'percent_num_words': 0
                })
                category['num_words'] += header['num_words']
                category['num_parts'] += header['num_parts']
                category['percent_num_words'] += header['percent_num_words']
                if header['longest_part_num_words'] > category['longest_part_num_words']:
                    category['longest_part_num_words'] = header['longest_part_num_words']
    print('Saving {} parts, {} headers, {} categories to {}'.format(len(parts), num_headers, num_categories, target_path))
    
    def _header_iterator():        
        for session_id in parts_session_ids:
            session = SESSIONS[session_id]
            if len(session['headers']) > 0:
                for header in session['headers'].values():
                    if (
                        (
                            len(header['categories']) == 0 
                            and header['num_parts'] == 1 
                            and header['percent_num_words'] < 0.5
                        ) or header['header'].strip() in [
                            'מוזמנים',
                            'סדר היום',
                            'מנהלת הועדה',
                            'מנהלת הוועדה'   
                        ]
                    ):
                        pass
                    else:
                        yield header
                        
    def _categories_iterator():
        for session_id in parts_session_ids:
            session = SESSIONS[session_id]
            if len(session['categories']) > 0:
                yield from session['categories'].values()

    Flow(
        iter(parts),
        _header_iterator(),
        _categories_iterator(),
        update_resource('res_1', name='parts', path='parts.csv'),
        update_resource('res_2', name='headers', path='headers.csv'),
        update_resource('res_3', name='categories', path='categories.csv'),
        dump_to_path(target_path)
    ).process()    
    for session_id in parts_session_ids:
        SESSIONS[session_id]['headers'] = None
        SESSIONS[session_id]['headers_stats'] = None
        SESSIONS[session_id]['categories'] = None


def process_part_group_subkeys(part_group_key, part_group_subkeys):
    print('processing part_group {} with {} subkeys'.format(part_group_key, len(part_group_subkeys)))
    all_rows = []
    for i, session_ids in enumerate(part_group_subkeys.values()):
        print('processing subkey {} with {} sessions'.format(i, len(session_ids)))
        session_ids_sql = ','.join([str(session_id) for session_id in session_ids])
        for row in stream_from_db("""
            select 
                sessions."CommitteeSessionID",
                sessions."KnessetNum",
                sessions.committee_name,
                parts."part_index",
                parts.header,
                parts.body_num_words,
                parts.body_num_words / parts_sum_words.sum_words * 100 percent_num_words,
                parts.part_categories,
                parts.name_role,
                parts.mk_individual_faction,
                sessions."StartDate",
                sessions."Note",
                sessions.topics,
                sessions.bill_names,
                sessions.bill_types
            from 
                people_committees_meeting_speaker_stats parts, 
                committees_kns_committeesession sessions,
                (
                    select
                        parts."CommitteeSessionID",
                        sum(parts."body_num_words") sum_words
                    from
                        people_committees_meeting_speaker_stats parts
                    where "CommitteeSessionID" in ({})
                    group by parts."CommitteeSessionID"
                ) parts_sum_words
            where 
                parts."CommitteeSessionID" in ({})
                and parts."CommitteeSessionID" = sessions."CommitteeSessionID"
                and parts_sum_words."CommitteeSessionID" = sessions."CommitteeSessionID"
            order by sessions."KnessetNum", sessions.committee_name, sessions."StartDate", parts.part_index
        """.format(session_ids_sql, session_ids_sql)):
            all_rows.append(row)
            stats['num_part_rows'] += 1
        if LIMIT_ROWS_PER_PART_GROUP and len(all_rows) > LIMIT_ROWS_PER_PART_GROUP:
            break
    print('loaded {}/{} part rows'.format(len(all_rows), stats['num_part_rows']))
    _dump_sessions_parts_to_path(part_group_key, all_rows)
        

def _is_part_group_updated(part_group_key):
    datapackage_json_path = '{}datapackage.json'.format(_get_part_group_target_path(part_group_key))
    if os.path.exists(datapackage_json_path):
        with open(datapackage_json_path) as f:
            datapackage = json.load(f)
            if datapackage.get('updated-datetime'):
                # datetime.datetime.strptime(datapackage['updated-datetime'], '%Y-%m-%d') - datetime.timedelta(days=5)
                return True
    return False
    

def _set_part_group_updated(part_group_key):
    datapackage_json_path = '{}datapackage.json'.format(_get_part_group_target_path(part_group_key))
    if os.path.exists(datapackage_json_path):
        with open(datapackage_json_path) as f:
            datapackage = json.load(f)
        datapackage['updated-datetime'] = datetime.datetime.now().strftime('%Y-%m-%d')
        with open(datapackage_json_path, 'w') as f:
            json.dump(datapackage, f)
    

def _part_groups_iterator(stats):
    for part_group_key, part_group_subkeys in SPEAKER_STATS_PART_GROUPS.items():
        stats['num_part_groups'] += 1
        if not _is_part_group_updated(part_group_key):
            yield part_group_key, part_group_subkeys

    
stats = defaultdict(int)        

for part_group_key, part_group_subkeys in _part_groups_iterator(stats):
    try:
        process_part_group_subkeys(part_group_key, part_group_subkeys)
        _set_part_group_updated(part_group_key)
    except Exception:
        traceback.print_exc()
        
print('Great Success!')
print(dict(stats))

processing part_group 15-הוועדה המיוחדת לענייני מחקר ופיתוח מדעי וטכנולוגי with 5 subkeys
processing subkey 0 with 1 sessions
processing subkey 1 with 6 sessions
processing subkey 2 with 4 sessions
processing subkey 3 with 8 sessions
processing subkey 4 with 1 sessions
loaded 7559/7559 part rows
dumping part_group_key 15-הוועדה המיוחדת לענייני מחקר ופיתוח מדעי וטכנולוגי with 7559 parts
Saving 7559 parts, 608 headers, 198 categories to data/detailed_speaker_parts/Knesset15/הוועדה המיוחדת לענייני מחקר ופיתוח מדעי וטכנולוגי/
processing part_group 15-הוועדה המיוחדת לקידום מעמד הילד with 4 subkeys
processing subkey 0 with 3 sessions
processing subkey 1 with 20 sessions
processing subkey 2 with 10 sessions
processing subkey 3 with 16 sessions
loaded 9344/16903 part rows
dumping part_group_key 15-הוועדה המיוחדת לקידום מעמד הילד with 9344 parts
Saving 9344 parts, 1176 headers, 333 categories to data/detailed_speaker_parts/Knesset15/הוועדה המיוחדת לקידום מעמד הילד/
processing part_group 15-הווע

Traceback (most recent call last):
  File "<ipython-input-11-2de1e3ba15a2>", line 88, in _dump_sessions_parts_to_path
    header['percent_num_words'] = round(header['num_words'] / header['session_num_words'] * 100, 2)
decimal.InvalidOperation: [<class 'decimal.DivisionUndefined'>]


processing part_group 20-הוועדה המשותפת לוועדת החוקה, חוק ומשפט ולוועדת הכספים לדיון בהצעת החוק לצמצום השימוש במזומן, התשע"ה-2015 (מ/945) with 1 subkeys
processing subkey 0 with 10 sessions
loaded 8350/2980415 part rows
dumping part_group_key 20-הוועדה המשותפת לוועדת החוקה, חוק ומשפט ולוועדת הכספים לדיון בהצעת החוק לצמצום השימוש במזומן, התשע"ה-2015 (מ/945) with 8350 parts
Saving 8350 parts, 300 headers, 134 categories to data/detailed_speaker_parts/Knesset20/הוועדה המשותפת לוועדת החוקה, חוק ומשפט ולוועדת הכספים לדיון /
processing part_group 20-הוועדה המשותפת לוועדת החוקה, חוק ומשפט ולוועדת הכספים לפי סעיף 13(ג) לפקודת המסים (גבייה) with 1 subkeys
processing subkey 0 with 1 sessions
loaded 29/2980444 part rows
dumping part_group_key 20-הוועדה המשותפת לוועדת החוקה, חוק ומשפט ולוועדת הכספים לפי סעיף 13(ג) לפקודת המסים (גבייה) with 29 parts
Saving 29 parts, 11 headers, 4 categories to data/detailed_speaker_parts/Knesset20/הוועדה המשותפת לוועדת החוקה, חוק ומשפט ולוועדת הכספים לפי סע/
proces

In [12]:
from dataflows import duplicate, add_field, delete_fields, update_resource
import os

def _process_speakers(rows):
    if rows.res.name == 'speakers':
        speakers = {}
        for header in rows:
            if header['header'] and len(header['header']) > 2:
                if header['header'].strip() in speakers:
                    speaker = speakers[header['header']]
                else:
                    speaker = speakers[header['header']] = {
                        **{k: header[k] for k in ['Note', 'topics', 'KnessetNum', 'committee_name']},
                        'num_meetings': 0,
                        'header': header['header'],
                        'bill_types': set(),
                        'bill_names': set(),
                        'categories': set(),
                        'name_roles': set(),
                        'mk_factions': set(),
                        **{k: 0 for k in ['num_words', 'num_parts', 'longest_part_num_words']}
                    }
                speaker['num_meetings'] += 1
                for bill_type in header['bill_types']:
                    speaker['bill_types'].add(bill_type)
                for bill_name in header['bill_names']:
                    speaker['bill_names'].add(bill_name)
                for category in header['categories']:
                    speaker['categories'].add(category)
                for name_role in header['name_roles']:
                    speaker['name_roles'].add(name_role)
                for mk_faction in header['mk_factions']:
                    speaker['mk_factions'].add(mk_faction)
                speaker['num_words'] += header['num_words']
                speaker['num_parts'] += header['num_parts']
                if speaker['longest_part_num_words'] == 0 or speaker['longest_part_num_words'] < header['longest_part_num_words']:
                    speaker['longest_part_num_words'] = header['longest_part_num_words']
        for speaker in speakers.values():
            speaker.update(bill_types=list(speaker['bill_types']),
                           bill_names=list(speaker['bill_names']),
                           categories=list(speaker['categories']),
                           name_roles=list(speaker['name_roles']),
                           mk_factions=list(speaker['mk_factions']),
                           )
            yield speaker
    else:
        yield from rows

        
def add_sessions(part_group_key):
    tmp = part_group_key.split('-')
    KnessetNum = tmp[0]
    committee_name = '-'.join(tmp[1:]).replace("'", "''")
    yield from stream_from_db("""
        select * from committees_kns_committeesession
        where "KnessetNum" = {}
        and committee_name = '{}'
    """.format(KnessetNum, committee_name))
    
        
stats = defaultdict(int)

for part_group_key, part_group_subkeys in SPEAKER_STATS_PART_GROUPS.items():
    stats['num_part_groups'] += 1
    datapackage_json_path = '{}datapackage.json'.format(_get_part_group_target_path(part_group_key))
    if os.path.exists(datapackage_json_path) and _is_part_group_updated(part_group_key):
        print('processing part_group_key {}'.format(part_group_key))
        with open(datapackage_json_path) as f:
            datapackage = json.load(f)
        if datapackage['count_of_rows'] > 0:
            Flow(
                load(datapackage_json_path, resources=['headers', 'parts', 'categories']),
                duplicate('headers', 'speakers', 'speakers.csv'),
                add_field('num_meetings', resources='speakers', type='number'),
                _process_speakers,
                delete_fields([
                    'Note', 'topics', 'StartDate', 'session_num_words', 'session_num_parts', 'percent_num_words',
                    'bill_types', 'bill_names'
                ], resources='speakers'),
                sort_rows('{num_words:09}', reverse=True, resources='speakers'),
                add_sessions(part_group_key),
                update_resource('res_5', name='sessions', path='sessions.csv'),
                # printer(tablefmt='html', num_rows=1),
                dump_to_path(_get_part_group_target_path(part_group_key))
            ).process()
        _set_part_group_updated(part_group_key)
        
print('Great Success!')
print(dict(stats))

processing part_group_key 15-הוועדה המיוחדת לענייני מחקר ופיתוח מדעי וטכנולוגי
processing part_group_key 15-הוועדה המיוחדת לקידום מעמד הילד
processing part_group_key 15-הוועדה לענייני ביקורת המדינה
processing part_group_key 15-הוועדה לקידום מעמד האישה
processing part_group_key 15-ועדת החוקה, חוק ומשפט
processing part_group_key 15-ועדת החינוך והתרבות
processing part_group_key 15-ועדת הכלכלה
processing part_group_key 15-ועדת הכנסת
processing part_group_key 15-ועדת הכספים
processing part_group_key 15-ועדת העבודה, הרווחה והבריאות
processing part_group_key 15-ועדת העלייה, הקליטה והתפוצות
processing part_group_key 15-ועדת הפנים ואיכות הסביבה
processing part_group_key 16-הוועדה המיוחדת לבחינת בעיית העובדים הזרים
processing part_group_key 16-הוועדה המיוחדת למאבק בנגע הסמים
processing part_group_key 16-הוועדה המיוחדת לקידום מעמד הילד
processing part_group_key 16-הוועדה המשותפת לדיון בהצעות חוק שעניינן אנטנות סלולריות
processing part_group_key 16-הוועדה המשותפת לוועדת החינוך ולוועדת הכספים
proce

## Create zip archive

In [13]:
%%bash
apk add zip
rm -f ./data/detailed_speaker_parts.zip
zip -r ./data/detailed_speaker_parts.zip ./data/detailed_speaker_parts

  adding: data/detailed_speaker_parts/ (stored 0%)
  adding: data/detailed_speaker_parts/Knesset20/ (stored 0%)
  adding: data/detailed_speaker_parts/Knesset20/הוועדה המשותפת לוועדת הכלכלה ולוועדת הכנסת לדיון בהצעת חוק ה/ (stored 0%)
  adding: data/detailed_speaker_parts/Knesset20/הוועדה המשותפת לוועדת הכלכלה ולוועדת הכנסת לדיון בהצעת חוק ה/categories.csv (deflated 83%)
  adding: data/detailed_speaker_parts/Knesset20/הוועדה המשותפת לוועדת הכלכלה ולוועדת הכנסת לדיון בהצעת חוק ה/headers.csv (deflated 91%)
  adding: data/detailed_speaker_parts/Knesset20/הוועדה המשותפת לוועדת הכלכלה ולוועדת הכנסת לדיון בהצעת חוק ה/speakers.csv (deflated 86%)
  adding: data/detailed_speaker_parts/Knesset20/הוועדה המשותפת לוועדת הכלכלה ולוועדת הכנסת לדיון בהצעת חוק ה/parts.csv (deflated 97%)
  adding: data/detailed_speaker_parts/Knesset20/הוועדה המשותפת לוועדת הכלכלה ולוועדת הכנסת לדיון בהצעת חוק ה/sessions.csv (deflated 86%)
  adding: data/detailed_speaker_parts/Knesset20/הוועדה המשותפת לוועדת הכלכלה ולוועד

bash: line 1: apk: command not found
